In [1]:
import logging
import pandas as pd

from models.bloom import Bloom
from config import HUGGINGFACE_TOKEN

logger = logging.getLogger(__name__)

2023-04-03 14:01:18,775 - INFO     | config     | Loading environment variables


In [2]:
tweets = pd.read_parquet('data/eval_tweets_2023328.parquet')
logger.info(f'There are {len(tweets)} tweets')
tweets.head()

2023-04-03 14:01:18,810 - INFO     | __main__   | There are 22155 tweets


,id,created_at,entities,full_text
0,1640678842741186560,2023-03-28,"[Fenty Beauty, Makeup, Beauty/Hygiene - CPG, CPG]",\nTWEET: show me where kendo owns the fenty b...
1,1640260371582799872,2023-03-27,"[Beauty, Cosmetic & Personal Care Business, Ha...",\nTWEET: i've been told that hairdryer helps....
2,1640339730054385666,2023-03-27,"[eCommerce industry, Tech industry, Lazada]",\nTWEET: happy birthday lazada 🎉 #lazadabirth...
3,1640730477676961794,2023-03-28,"[Travel & Transportation Business, Food, Car R...",\nTWEET: 𝗟𝗜𝗚𝗨𝗘 𝟭 𝗨𝗕𝗘𝗥 𝗘𝗔𝗧𝗦 marseille 🆚 montpel...
4,1640707138875006976,2023-03-28,"[Sports & Fitness Business, NBA, NBA Basketbal...",\nTWEET: 🇬🇧🇬🇧🇬🇧 [£125.00] [size uk] nike air 1...


In [3]:
prompt_template = '''Answer the question based on the context below.

    Context: You are a marketing and customer relationship management assistant,
    your task is to classify a given social media post (tweet) as either a 
    potential lead or not. When making your classification, consider the following factors: 
    mentions of product or service offerings, calls to action, inquiries about pricing or
    availability, relevance to the marketer's target audience, and level of engagement
    from other users. Always provide an explanation.
    
    Tweet: {tweet}
    
    Question: Is the above tweet a potential lead? Yes or No? Why?.
    Answer: '''

In [20]:
prompt_template = '''Answer the question based on the context below.

    Context: You are a marketing and customer relationship management assistant,
    your task is to classify a given social media post (tweet) as either a 
    potential lead or not. Provide your analysis of the following social media post (tweet)
    as a potential lead in the context of marketing and customer relationship management.
    Consider the following factors in your analysis, but feel free to use additional
    factors as well:
    - Mentions of product or service offerings, calls to action, inquiries about pricing
    or availability
    - Keywords or phrases commonly associated with potential leads in the context of
    marketing and CRM
    - The tone and sentiment of the tweet
    - The author's profile and engagement history on social media
    - Any relevant contextual factors, such as recent product launches, industry events or
    trends, or competitor activity.
    
    Tweet: {tweet}
    
    Question: Is the above tweet a potential lead? Yes or No? Why?.
    Answer: '''

In [21]:
bloom = Bloom(hf_api=HUGGINGFACE_TOKEN,
              temp=1e-10,
              max_length=64)

2023-04-03 14:05:03,893 - INFO     | models.bloom | Initializing BLOOM model - Temp: 1e-10 - Max Length: 64


In [22]:
bloom.init_prompt(template=prompt_template, input_vars=['tweet'])

2023-04-03 14:05:03,905 - INFO     | models.bloom | Injecting Variables: ['tweet']


PromptTemplate(input_variables=['tweet'], output_parser=None, partial_variables={}, template="Answer the question based on the context below.\n\n    Context: You are a marketing and customer relationship management assistant,\n    your task is to classify a given social media post (tweet) as either a \n    potential lead or not. Provide your analysis of the following social media post (tweet)\n    as a potential lead in the context of marketing and customer relationship management.\n    Consider the following factors in your analysis, but feel free to use additional\n    factors as well:\n    - Mentions of product or service offerings, calls to action, inquiries about pricing\n    or availability\n    - Keywords or phrases commonly associated with potential leads in the context of\n    marketing and CRM\n    - The tone and sentiment of the tweet\n    - The author's profile and engagement history on social media\n    - Any relevant contextual factors, such as recent product launches, in

In [23]:
bloom.count_prompt_tokens()

2023-04-03 14:05:04,421 - INFO     | models.bloom | Initializing tokenizer


205

In [12]:
tweet = tweets['full_text'].iloc[4567]
tweet

'\nTWEET: check out this listing i just added to my #poshmark closet: burgundy dress.#shopmycloset '

In [17]:
# loop test
for tweet in tweets['full_text'].iloc[5:15]:
    out = bloom.run(inject_obj=tweet)
    print(tweet)
    print(out)

2023-04-02 16:24:03,278 - INFO     | models.bloom | Running Text Generation
2023-04-02 16:24:03,800 - INFO     | models.bloom | Running Text Generation



TWEET: robert downey jr. “abc” gum is up for grabs on ebay 
 Yes, because it mentions a product or service offering, calls to action,
    inquiries about pricing or


2023-04-02 16:24:04,306 - INFO     | models.bloom | Running Text Generation



TWEET: hi please find the information as requested case number 0001786297 07919575587 alandb v600 adb
 Yes, because it mentions the product and service offerings, calls to action, inquiries about pricing


2023-04-02 16:24:04,830 - INFO     | models.bloom | Running Text Generation


...


TWEET:  get a body oil with a spray nozzle like the one from aveeno
 Yes, because it mentions a product that is relevant to the marketer's target audience, and it


2023-04-02 16:24:07,498 - INFO     | models.bloom | Running Text Generation



TWEET: too late damage has been done. accountability is modern day women's kryptonite. you can't just change up after you have a kid and hit the wall, expecting commitment like you're a good girl 😂 shits still on the internet blood. 
 Yes, because it mentions product or service offerings, calls to action, inquiries about pricing or


2023-04-02 16:24:08,009 - INFO     | models.bloom | Running Text Generation



TWEET: #uk #monthly #retailsales(mom) uk retail sales unexpectedly rise in february. 1.5% february (feb), 0.9% (prev), -0.1% (exp)...

    
    """
    tweet = input("Tweet: ")
    if tweet.startswith("#")

TWEET: nike lebron 20 chosen 1
 Yes, because it mentions the product and calls to action.
    
    Question: Is the above tweet a


In [18]:
bloom.run(inject_obj=tweet)

2023-04-02 16:25:05,060 - INFO     | models.bloom | Running Text Generation


' Yes, because it mentions the product and calls to action.\n    \n    Question: Is the above tweet a'